In [ ]:
import os
import torch
import configparser
from data_loader.era5 import ERA5
from model.Model import Model
import datetime

# Setting

In [ ]:
config = configparser.ConfigParser()
config._interpolation = configparser.ExtendedInterpolation()
config.read('./setting.config')
torch.manual_seed(42)

In [ ]:
is_train = str(config['model']['is_train']).lower() == 'true'
model_save_folder = str(config['model']['model_save_folder'])

# Data Loader

In [ ]:
print("loading era5....")
era5 = ERA5(config)
train_dataloader = era5.train_loader()
valid_dataloader = era5.validate_loader()
test_loader = era5.test_loader()

# Train

In [ ]:
if is_train == True:

    # new training
    if model_save_folder == '':
        model_save_folder = './save-{}'.format(datetime.datetime.now().strftime('%Y%m%d_%H%M%S_%f'))
        model = Model(config, era5=era5, model_save_folder=model_save_folder)
        if os.path.exists(model_save_folder) == False:
            os.makedirs(model_save_folder)
            model.setup_logger(model_save_folder)
        print("start training....")
        model.train(train_dataloader, valid_dataloader)
    else:
        # retrain
        model = Model(config, era5=era5, model_save_folder=model_save_folder)
        start_epoch, min_loss = model.load(model_save_folder)
        model.setup_logger(model_save_folder)
        print("start retraining....")
        model.train(train_dataloader, valid_dataloader, 
                    start_epoch=start_epoch, min_loss=min_loss)
        
    print("start testing....")
    model_save_path = model_save_folder+'/best_val.pth'
    model.evaluate_test(test_loader, model_save_path=model_save_path)

# Test

In [ ]:
if is_train == False:
    print("start testing....")

    model = Model(config, era5=era5, model_save_folder=model_save_folder)
    
    model.setup_logger(model_save_folder)
    model_save_path = '{}/best_val.pth'.format(model_save_folder)
    model.evaluate_test(test_loader, model_save_path=model_save_path, builtins_print=True)